In [ ]:
import qiskit
import qiskit_ibm_runtime
import numpy as np
from numpy import pi
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit.circuit import Parameter

In [ ]:
service = QiskitRuntimeService.save_account(token, instance="*", overwrite="True")

In [ ]:
γ = Parameter('γ')
η = Parameter('η')
β = Parameter('β')

X = SparsePauliOp("X")
Z = SparsePauliOp("Z")

qc_sym = QuantumCircuit(7)

qc_sym.h([0,1])
qc_sym.cx(0,5)
qc_sym.cx(1,6)
qc_sym.barrier()

ZZZ = PauliEvolutionGate(Z^Z^Z, time=β*γ)
ZX = PauliEvolutionGate(Z^X, time=β*η)

qc_sym.h([2,3,4])

qc_sym.append(ZZZ,[0,1,2])
qc_sym.append(ZX,[0,3])
qc_sym.append(ZX,[1,4])

qc_sym.append(ZZZ,[0,1,2])
qc_sym.append(ZX,[0,3])
qc_sym.append(ZX,[1,4])


qc_sym.sx([2,3,4])

qc_sym.cx(2,0)
qc_sym.cz(3,0)
qc_sym.cz(4,1)

qc_sym.draw("mpl",scale=0.5)

In [ ]:
sim = AerSimulator(method="statevector")

In [ ]:
H_sym = SparsePauliOp(["IIIIIZZ","IIIIIXI","IIIIIIX"], coeffs=[γ,η,η])

In [ ]:
H_fix = H_sym.assign_parameters({γ:pi/4,η:pi/4})
H_fix = SparsePauliOp(H_fix.paulis, coeffs=np.asarray(H_fix.coeffs, dtype=np.complex128)) #Forcing correct assignment

In [ ]:
vals = np.linspace(0.0, 0.4, 11)

In [ ]:
pm = generate_preset_pass_manager(backend=sim, optimization_level=3)

In [ ]:
results = []  # elenco di dict per tenere tutto allineato
for val in vals:
    qc = qc_sym.assign_parameters({γ: pi/4, η: pi/4, β: -val})
    isa_qc = pm.run(qc)                 # circuito compilato per il backend
    isa_H = H_fix.apply_layout(isa_qc.layout)  # H adattato al layout del circuito
    results.append({"beta": val, "isa_qc": isa_qc, "isa_H": isa_H})

In [ ]:
estimator = Estimator(mode=sim, options={"default_shots": 10000})

In [ ]:
trotter_steps = 2

In [ ]:
approx = []
for item in results:  # results proviene dal codice precedente
    beta = item["beta"]
    isa_qc = item["isa_qc"]
    isa_H = item["isa_H"]

    res = estimator.run([(isa_qc, isa_H)]).result()[0]
    ev = res.data.evs
    approx.append((beta*2*trotter_steps, ev))
    print(f"β = {beta*2*trotter_steps:.3f}, Expectation value = {ev}")

In [ ]:
# --- Parametri (usiamo gli stessi del tuo setup) ---
gamma = pi/4
eta = pi/4

# --- Matrici di Pauli e identità ---
I = np.array([[1, 0],
              [0, 1]], dtype=complex)
X = np.array([[0, 1],
              [1, 0]], dtype=complex)
Z = np.array([[1, 0],
              [0,-1]], dtype=complex)

# --- Costruzione di H = gamma ZZ + eta (XI + IX) ---
ZZ = np.kron(Z, Z)
XI = np.kron(X, I)
IX = np.kron(I, X)
H = gamma * ZZ + eta * (XI + IX)

eps, V = np.linalg.eigh(H)

def thermal_average_H(beta: float) -> float:

    w = np.exp(-beta * eps)
    Z = np.sum(w)
    return float(np.sum(eps * w) / Z)

beta_values = np.linspace(0., 1.6, 11)

thermal_averages = np.array([thermal_average_H(b) for b in beta_values])

for b, e in zip(beta_values, thermal_averages):
    print(f"beta = {b:.3f} -> <H>_beta = {e:.10f}")

In [ ]:
def exp_func(beta, a, b, c):
    return a * np.exp(-b * beta) + c

In [ ]:
gs = np.min(np.linalg.eigvalsh(H))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# beta_values, thermal_averages, expectation_values già definiti
betas_circ, evs_circ = zip(*approx)

# Funzione esponenziale
def exp_func(beta, a, b, c):
    return a * np.exp(-b * beta) + c

# Fit esponenziale
params_th, _ = curve_fit(exp_func, beta_values, thermal_averages, p0=(1,1,0))
params_circ, _ = curve_fit(exp_func, betas_circ, evs_circ, p0=(1,1,0))

beta_smooth = np.linspace(min(beta_values), max(beta_values), 200)
thermal_fit = exp_func(beta_smooth, *params_th)
circuit_fit = exp_func(beta_smooth, *params_circ)

plt.figure(figsize=(6,4),dpi=140)

plt.scatter(beta_values, thermal_averages, color='skyblue', label='exact')
plt.scatter(betas_circ, evs_circ, color='lightgreen', label='approx')

plt.plot(beta_smooth, thermal_fit, color='skyblue', linestyle='-')
plt.plot(beta_smooth, circuit_fit, color='lightgreen', linestyle='--')

plt.axhline(y=gs, color='gray', linestyle=':', linewidth=2, label='ground')

ymin = min(np.min(thermal_averages), np.min(evs_circ), gs)
ymax = max(np.max(thermal_averages), np.max(evs_circ))
plt.ylim(ymin - 0.1 * abs(ymin), ymax)

plt.xlabel(r'$\beta$', fontsize=12)
plt.ylabel(r'$\langle H \rangle $', fontsize=12)
plt.grid(True, alpha=0.5)
plt.legend()
plt.tight_layout()
plt.show()